In [1]:
%cd ..

/home/aymeric/Documents/DAC/NeuralNetworksDIY


In [2]:
from src.convolution import *
from utils.mltools import *

def load_usps(fn):
    with open(fn, "r") as f:
        f.readline()
        data = [[float(X) for X in l.split()] for l in f if len(l.split()) > 2]
    tmp = np.array(data)
    return tmp[:, 1:], tmp[:, 0].astype(int)


def get_usps(l, X, y):
    if type(l) != list:
        resx = X[y == l, :]
        resy = y[y == l]
        return resx, resy
    tmp = list(zip(*[get_usps(i, X, y) for i in l]))
    tmpx, tmpy = np.vstack(tmp[0]), np.hstack(tmp[1])
    return tmpx, tmpy

alltrainx, alltrainy = load_usps("data/USPS_train.txt")
alltestx, alltesty = load_usps("data/USPS_test.txt")


def load_one_class(number):
    X_train, y_train = get_usps(number, alltrainx, alltrainy)
    X_test, y_test = get_usps(number, alltestx, alltesty)
    y_train = np.where(y_train == number, -1, 1).reshape(-1, 1)
    y_test = np.where(y_test == number, -1, 1).reshape(-1, 1)
    return X_train, y_train, X_test, y_test


def load_two_classes(neg, pos):
    X_train, y_train = get_usps([neg, pos], alltrainx, alltrainy)
    X_test, y_test = get_usps([neg, pos], alltestx, alltesty)
    y_train = np.where(y_train == neg, -1, 1).reshape(-1, 1)
    y_test = np.where(y_test == neg, -1, 1).reshape(-1, 1)
    return X_train, y_train, X_test, y_test


In [3]:
X_train, y_train, X_test, y_test = load_two_classes(6, 9)


In [4]:
alltrainx.shape

(7291, 256)

In [5]:
import numpy as np
X_train[:,:,np.newaxis].shape

(1308, 256, 1)

In [6]:
conv = Conv1D(3, 1, 1)

In [7]:
conv.forward(X_train[:,:,np.newaxis]).shape

(1308, 254, 1)

## Vérif Conv

In [8]:
import torch
import torch.nn as nn


In [9]:
convtest = nn.Conv1d(3, 1, 3, stride=1, bias=False)
# chan_in, chan_out, k_size

In [10]:
input = torch.randn(20, 3, 50) # N, C_IN, L_IN
convtest(input).shape

torch.Size([20, 1, 48])

In [11]:
input.numpy().shape

(20, 3, 50)

In [12]:
np.transpose(convtest.weight.data.numpy()).shape

(3, 3, 1)

In [13]:
conv = Conv1D(3, 3, 1, stride=1)
conv._chan_in

3

In [14]:
t = conv.forward(input.numpy().reshape((20, 50, 3)))

In [15]:
t.shape

(20, 48, 1)

## Vérif MaxPool

In [24]:
maxpool = MaxPool1D(2, 2)
t = maxpool.forward(input.numpy().reshape((20, 50, 3)))
t

array([[[ 1.36679947,  1.36679947,  1.36679947],
        [ 1.76574707,  1.76574707,  1.76574707],
        [ 1.64782441,  1.64782441,  1.64782441],
        ...,
        [ 1.09715641,  1.09715641,  1.09715641],
        [ 0.89641809,  0.89641809,  0.89641809],
        [ 2.34088182,  2.34088182,  2.34088182]],

       [[ 1.10620284,  1.10620284,  1.10620284],
        [ 1.68932664,  1.68932664,  1.68932664],
        [ 2.80534434,  2.80534434,  2.80534434],
        ...,
        [ 1.29367065,  1.29367065,  1.29367065],
        [ 0.77202016,  0.77202016,  0.77202016],
        [ 0.58672637,  0.58672637,  0.58672637]],

       [[ 1.13395751,  1.13395751,  1.13395751],
        [ 1.27314234,  1.27314234,  1.27314234],
        [ 1.76051867,  1.76051867,  1.76051867],
        ...,
        [ 1.49187005,  1.49187005,  1.49187005],
        [ 0.62750763,  0.62750763,  0.62750763],
        [ 1.27847016,  1.27847016,  1.27847016]],

       ...,

       [[ 0.5015828 ,  0.5015828 ,  0.5015828 ],
        [ 1

In [27]:
maxpooltorch = nn.MaxPool1d(2, 2)
maxpooltorch(input).shape

torch.Size([20, 3, 25])

In [26]:
AvgPool1D(2, 2).forward(input.numpy().reshape((20, 50, 3))).shape

(20, 25, 3)